<a href="https://colab.research.google.com/github/izzZk-hub/limpieza_datos_y_eda/blob/main/1_limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza de datos

La mayoría de la gente piensa que ser científico de datos significa estar corriendo modelos avanzados de Machine Learning todo el tiempo. La realidad es muy distinta:

![Tiempo de un científico de datos](https://github.com/izzZk-hub/limpieza_datos_y_eda/blob/main/img/datascientist_time.jpeg?raw=1)

La gran mayoría del tiempo se va **limpiando y organizando** los datos con los que queremos trabajar.

Los científicos de datos utilizamos herramientas como Pandas, Numpy, Matplotlib y Seaborn para limpiar los datos con los que queremos hacer algo.

## ETL

Un tipo de tarea que realizamos con gran frecuencia los científicos de datos son los **ETL**.

![Proceso de ETL](https://github.com/izzZk-hub/limpieza_datos_y_eda/blob/main/img/etl.png?raw=1)

ETL es un acrónimo que significa Extract, Transform, Load. Es un proceso que se utiliza para extraer datos de una fuente, transformarlos en un formato que sea adecuado para el análisis y cargarlos en una base de datos o algún otro sistema de almacenamiento.

## Ejemplo práctico

Los datos que usaremos para esta limpieza y nuestro siguiente análisis son datos de incidencia delictiva en nuestro país.

La iniciativa de datos abiertos del gobierno de México nos proporciona datos de incidencia delictiva desde 2015 hasta la fecha. Los datos se actualizan todos los meses y se pueden descargar desde el siguiente enlace: https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva

---

Como podemos ver en el portal, se proporcionan los datos tanto a nivel estatal como a nivel municipal. En este caso, utilizaremos los datos a nivel estatal.

Descarguemos los datos y guardemos el archivo CSV en la carpeta data con el nombre `datos_delitos.csv`.

---

Ahora leamos el archivo CSV y veamos cómo se ven los datos.

Primero que nada, importemos pandas

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data/datos_delitos.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/datos_delitos.csv'

Aquí tenemos un error muy común que suele ocurrir cuando un archivo se guarda en una computadora con cierto "encoding".

Un encoding es una tabla que relaciona un número con un carácter. Por ejemplo, en la tabla ASCII, el número 65 corresponde a la letra "A".

Si el archivo que estamos leyendo fue guardado con un encoding distinto al que pandas espera, nos arrojará un error.

Para solucionar esto, podemos utilizar el parámetro `encoding` de la función `pd.read_csv()` y especificar el encoding correcto.

¿Pero cómo sabemos con qué encoding cuenta el archivo? Si no sabemos el encoding de un archivo, primero probamos con utf-8; si falla, usamos ISO-8859-1 o latin1.

La realidad es que la gran mayoría de los archivos los encontrarán en encoding utf-8 y Pandas no va a dar ningún error. Aquí estamos teniendo este problema porque la computadora que utilizan para generar este archivo uso un encoding diferente a utf-8.

En México, por lo general, si un archivo no está en utf-8, lo más seguro es que esté en `ISO-8859-1` o `latin1`.

Intentemos cargar el archivo especificando el encoding `ISO-8859-1`.

In [ ]:
df = pd.read_csv('data/datos_delitos.csv', encoding='ISO-8859-1')
df.head()

Y vemos que ya podemos leer correctamente el archivo.


¿Existe alguna forma de verificar el encoding de un archivo sin tener que estar adivinando? Sí, se puede verificar el encoding de un archivo usando librerías como chardet en Python, que detecta automáticamente el encoding más probable sin tener que adivinar.

ChatGTP generó el siguiente código:

In [ ]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read()
    result = chardet.detect(rawdata)
    return result

file_path = './data/datos_delitos.csv'
encoding_info = detect_encoding(file_path)
print(f"Detected encoding: {encoding_info['encoding']}")

Sigamos...

In [ ]:
df.tail(3)

In [ ]:
df.info()

Bien. Están muy bien los datos. Sin embargo, tenemos un problema con el que es muy común encontrarnos.

Resulta que el formato en el que está el archivo es fácil entender por seres humanos:


```markdown
| Año      | Entidad  | Enero    | Febrero  | Mes X    |
|----------|----------|----------|----------|----------|
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
```

Vemos que tenemos los meses como encabezados. Es decir, el archivo está tratando cada mes como si fuera una variable.

Nosotros como científicos de datos estamos más interesados en conjuntos de datos que no estén en este formato de "resumen" o "tabla dinámica". Para nosotros, lo ideal sería que cada mes fuera simplemente una observación más en nuestro conjunto de datos. Es decir, queremos transformar la tabla de arriba en:

```markdown
| Año      | Entidad  | Mes        |
|----------|----------|------------|
|   ...    |   ...    |   Enero    |
|   ...    |   ...    |   Febrero  |
|   ...    |   ...    |   Marzo    |
|   ...    |   ...    |   Abril    |
|   ...    |   ...    |   Mes X    |
```

A este tipo de formato le llamos "formato largo de datos".

Haremos las siguientes limpiezas:
* Transformar los nombres de las columnas para que no tengan caracteres especiales y estén siempre en minúsculas
* Convertir el dataset a un formato de datos "largo"

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
len(df.columns)

### Limpiar nombres de columnas

In [ ]:
def limpiar_columnas(df):
    columnas_limpias = []
    for col in df.columns:
        # convertir a minusculas, reemplazar espacios por guiones bajos y eliminar caracteres especiales
        col = col.lower().replace(" ", "_").replace("ñ", "ni").replace(".", "").replace("á", "a").replace("é", "e").replace("í","i").replace("ó", "o").replace("ú", "u")
        columnas_limpias.append(col)

    df.columns = columnas_limpias

    return df


In [ ]:
df.head(2)

In [ ]:
df = limpiar_columnas(df)

In [ ]:
df.head(3)

Muy bien. Ahora lo que queremos hacer es quitar algunas columnas. Nos interesan nada más las siguientes:

In [ ]:
df[['anio', 'entidad']]

In [ ]:
df = df[['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'subtipo_de_delito', 'modalidad','enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']]
df.head(2)

### Formato largo de datos

Ahora, usaremos el método `melt` para convertir las columnas a observaciones.

Queremos convervar las coumnas:
* anio
* clave_ent
* entidad
* tipo_de_delito
* subtipo_de_delito
* modalidad

El resto de las columnas las vamos a juntar en una nueva columna llamada "nombre_mes" y sus valores los vamos a sumar en otra llamada "frecuencia"

In [ ]:
print("Shape ", df.shape)

In [ ]:
datos_long = df.melt(id_vars=['anio', 'clave_ent', 'entidad','tipo_de_delito', 'subtipo_de_delito', 'modalidad'], var_name='nombre_mes', value_name='frecuencia')

In [ ]:
print("Shape: ", datos_long.shape)

In [ ]:
datos_long.head(5)

In [ ]:
datos_long.info()

Supongamos que para este análisis, no nos importan los niveles subtipo de delito y modalidad. O sea, no queremos tener la distinción entre homicidios dolosos y culposos (sé que son bastante diferentes, pero simplifiquemos nuestro ejemplo).

Vamos a agrupar nuestro dataframe por anio, clave_ent, entidad, tipo_de_delito y nombre_mes. Esto hará que todos los tipos de homicidios se sumen al tipo "homicidio" o todos los tipos de robo de vehículo (con o sin violencia) se sumen a "robo de vehículo".

In [ ]:
datos_long = datos_long.groupby(['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'nombre_mes'])['frecuencia'].sum().reset_index()

In [ ]:
datos_long[datos_long.tipo_de_delito == 'Robo'].sample(15)

Mostremos todos los estados y su respectiva clave

In [ ]:
datos_long[['clave_ent', 'entidad']].drop_duplicates().sort_values('entidad')

Ahora Veamos todos los datos de delitos de una entidad en específico. Por ejemplo, Nuevo león

In [ ]:
datos_long[datos_long['clave_ent'] == 19].sample(15)

In [ ]:
datos_long[(datos_long['clave_ent'] > 19) &  (datos_long['clave_ent'] < 24) & (datos_long['tipo_de_delito'] == 'Homicidio')].sample(15)

### Valores de fechas

Finalmente, queremos tener una columna "fecha". Actualmente tenemos el año y el nombre del mes, pero no tenemos como tal una columna que tenga un tipo de dato fecha. Eso hace que filtrar por fecha sea complicado.

Por ejemplo, si queremos conocer todos los homicidios de Oaxaca en enero 2024, haríamos lo siguiente:

In [ ]:
datos_long[
    (datos_long['clave_ent'] == 20) &
    (datos_long['tipo_de_delito'] == 'Homicidio') &
    (datos_long['anio'] == 2024) &
    (datos_long['nombre_mes'] == 'enero')
]

Creemos una columna de fecha.

Primero tenemos que convertir el nombre de mes a un número, en donde 1 es enero, 2 febrero, etc.

In [ ]:
datos_long.sample(2)

In [ ]:
datos_long["nueva_columna"] = "dato vacío"
datos_long.sample(6)

In [ ]:
# Diccionario de ayuda para convertir
meses = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12
}

datos_long['mes'] = datos_long['nombre_mes'].map(meses)
datos_long.sample(5)

In [ ]:
datos_long["frecuencia_mas_10"] = datos_long["frecuencia"] + 10
datos_long.sample(5)

In [ ]:
datos_long["anio_mes"] = datos_long["anio"].astype(str) + datos_long["mes"].astype(str)
datos_long.sample(6)

In [ ]:
# yyyy-mm-dd, yy-mm-dd, yymmdd, yyyy/dd/mm

In [ ]:
# Agregamos la columna de fecha juntando el año y el mes
datos_long['fecha'] = pd.to_datetime(datos_long['anio'].astype(str) + datos_long['mes'].astype(str), format='%Y%m')
datos_long.sample(5)

In [ ]:
# Eliminamos las columnas que ya no necesitamos
datos_long = datos_long.drop(columns=['nueva_columna'])
datos_long.sample(5)

Veamos los homicidios en oaxaca de enero 2024 a la fecha

In [ ]:
datos_long[
    (datos_long.tipo_de_delito == "Homicidio") &
    (datos_long.clave_ent == 20) &
    (datos_long.fecha >= '2024-01-01')
].sort_values('fecha')

In [ ]:
datos_finales = datos_long[['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'nombre_mes', 'fecha', 'frecuencia']]
datos_finales.head(2)

Ya que tenemos muestros datos bien estructurados, los podemos guardar en nuestra computadora. Los guardaremos con el nombre "delitos.csv"

In [ ]:
datos_finales.to_csv('data/delitos.csv', index=False)